In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys
import numpy as np
from opening_hours import OpeningHours
import re

In [2]:
df = pd.read_csv('osm_places_amenity.csv')
# TODO: You're removing the wrong index on any list with more than one thing to remove [-i-1] indexing doesn't work
# STORY: You're currently creating the remove days off function

C:\Users\Daniel\AppData\Local\Temp\ipykernel_21052\2753785423.py:1: DtypeWarning: Columns (47,70) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('osm_places_amenity.csv')


In [3]:
original_hours = df['opening_hours'].astype(str).unique()

In [4]:
# Split by ;
hours = [[hour.strip() for hour in hrs.split(';')] for hrs in original_hours ]

In [5]:
# Change shortenings of some days for the OpeningHours parser to work.
day_map = {
    "Mo":"Mon",
    "Tu":"Tue",
    "We":"Wed",
    "Fr": "Fri"
}


for i, hrs in enumerate(hours):
    for j, _ in enumerate(hrs):
        for key, val in day_map.items():
            hours[i][j] = hours[i][j].replace(key, val)

In [6]:
# Split by commas
hours = [[[hr_1.strip()  for hr_1 in hr.split(',')] for hr in hrs]for hrs in hours]

In [7]:
print(hours)

[[['Mon-Su 11:00-22:00']], [['nan']], [['Mon-Th 12:00-22:00'], ['Fri-Sa 12:00-23:00'], ['Su 12:00-22:00']], [['Mon-Fri 09:00-13:00', '15:00-18:00']], [['05:00-24:00']], [['week 1-20', '35-52 Mon-Th 08:30-21:00'], ['Fri 08:30-19:00'], ['Sa 09:00-18:00'], ['Su 14:00-19:00'], ['PH off']], [['Mon-Th 11:00-18:00'], ['Fri-Sa 10:00-17:00'], ['Su off']], [['Mon-Su 11:00-04:00']], [['Mon-Su 12:00-23:00']], [['Mon-Fri 07:00-19:00'], ['Sa-Su 08:00-16:00']], [['Mon-Fri 12:00-21:00'], ['Sa-Su 11:00-21:00']], [['Mon-Th 17:30-23:30'], ['Fri 17:30-01:00'], ['Sa 11:00-15:30', '17:30-01:00'], ['Su 11:00-15:30', '17:00-23:30']], [['Mon-Th 16:00-02:00'], ['Fri-Sa 16:00-04:00'], ['Su 16:00-02:00']], [['Mon-Th 11:30-15:00', '17:00-22:00'], ['Fri-Sa 11:30-15:00', '17:00-22:30'], ['Su 11:30-15:00', '17:00-22:00']], [['Mon-Su 15:00-23:00']], [['Mon-Su 09:00-07:00']], [['Mon-Su 12:00-22:00']], [['Mon-Th 10:30-15:30', '17:00-22:00'], ['Fri 10:30-23:00'], ['Sa 12:00-23:00'], ['Su 12:00-22:00']], [['Tue-Sa 17:00-2

In [8]:

# Remove public holidays and summer holidays
# All have PH/SH in them
for i, hrs in enumerate(hours):
    for j, hr in enumerate(hrs):
        for k, hr_1 in enumerate(hr[::-1]):
            if re.search("PH|SH", hr_1):
                del hours[i][j][-k-1]

In [9]:
# Functions for determining if statements contain days
day_re = r"(Mon|Tue|Wed|Th|Fri|Sa|Su)(-(Mon|Tue|Wed|Th|Fri|Sa|Su))?"
time_re = r"([01]\d|2[0-9]):([0-5]\d)(-([01]\d|2[0-9]):([0-5]\d))?|off|closed"
time_range_re = r"([01]\d|2[0-9]):([0-5]\d)(-([01]\d|2[0-9]):([0-5]\d))|off|closed"
def contains_days(string):
    if re.search(day_re, string):
        return True
    return False

def contains_times(string):
    if re.search(time_re, string):
        return True
    return False



In [10]:
for i,hrs in enumerate(hours):
    # If a time after a comma is missing the days, then we must give it the days from the comma before 
    # e.g. Mon-Th 11:00-12:00, 13:00-15:00 =>  Mon-Th 11:00-12:00, Mon-Th 13:00-15:00
    for j, hr in enumerate(hrs):
        prev_days = None
        for k, hr_1 in enumerate(hr):
            if contains_days(hr_1):
                prev_days = re.search(day_re, hr_1).group()
            elif prev_days:
                print("HERE", hours[i][j][k], prev_days)
                hours[i][j][k] = prev_days + " " + hours[i][j][k]
                print("HERE", hours[i][j][k], prev_days)

    
        # # If a day before a comma is missing the hours, then we must give it the days from the comma after 
        # # e.g. Mon-Th,Fri 13:00-15:00 =>  Mon-Th 13:00-15:00, Fri 13:00-15:00
        prev_hours = None
        for k, hr_1 in enumerate(hr[::-1]):
            if contains_times(hr_1):
                prev_hours = re.search(time_re, hr_1).group()
            elif prev_hours:
                print("HERE", hr_1, prev_hours)
                hours[i][j][-k-1] = hours[i][j][-k-1] + " " + prev_hours
                print("HERE", hr_1, prev_hours)


HERE 15:00-18:00 Mon-Fri
HERE Mon-Fri 15:00-18:00 Mon-Fri
HERE week 1-20 08:30-21:00
HERE week 1-20 08:30-21:00
HERE 17:30-01:00 Sa
HERE Sa 17:30-01:00 Sa
HERE 17:00-23:30 Su
HERE Su 17:00-23:30 Su
HERE 17:00-22:00 Mon-Th
HERE Mon-Th 17:00-22:00 Mon-Th
HERE 17:00-22:30 Fri-Sa
HERE Fri-Sa 17:00-22:30 Fri-Sa
HERE 17:00-22:00 Su
HERE Su 17:00-22:00 Su
HERE 17:00-22:00 Mon-Th
HERE Mon-Th 17:00-22:00 Mon-Th
HERE Sa 12:00-23:00
HERE Sa 12:00-23:00
HERE Sa 14:00-23:00
HERE Sa 14:00-23:00
HERE Fri 12:00-24:00
HERE Fri 12:00-24:00
HERE Sa 11:00-24:00
HERE Sa 11:00-24:00
HERE Fri 11:00-23:30
HERE Fri 11:00-23:30
HERE Mon 06:00-22:00
HERE Mon 06:00-22:00
HERE Mon-Th 08:00-03:00
HERE Mon-Th 08:00-03:00
HERE Fri 08:00-04:00
HERE Fri 08:00-04:00
HERE Fri 12:00-04:00
HERE Fri 12:00-04:00
HERE Mon-Th 08:00-23:00
HERE Mon-Th 08:00-23:00
HERE Fri 08:00-24:00
HERE Fri 08:00-24:00
HERE Sa 10:00-24:00
HERE Sa 10:00-24:00
HERE 17:00-22:00 Mon-Su
HERE Mon-Su 17:00-22:00 Mon-Su
HERE Sa 16:00-23:00
HERE Sa 16:

In [11]:
# Now convert the closing time to a reasonable time if >23:57
# e.g. 11:00-24:00 => 11:00-00:00, 11:00-28:00 => 11:00-04:00
overtime_re = r"([01]\d|2[0-9]):([0-5]\d)(-(2[4-9]):([0-5]\d))"
for i, hrs in enumerate(hours):
    for j, hr in enumerate(hrs):
        for k, hr_1 in enumerate(hr):
            # Find overtime hours
            search = re.search(overtime_re, hr_1)
            if re.search(overtime_re, hr_1):
                before = hr_1[:search.start()+6]
                # Subtract 24 from the time that's >24
                hour_reduced = int(hr_1[search.start() + 6:search.start()+ 8]) - 24
                after = hr_1[search.start()+8:]
                # Replace string with new formatted string
                hours[i][j][k] = f"{before}0{hour_reduced}{after}"
            

In [12]:
# Replace + symbols with -00:00 in the case where it's not already a time range 
# i.e. 11:00+ => 11:00-00:00
# But 11:00-23:00+ => 11:00-23:00
for i, hrs in enumerate(hours):
    for j, hr in enumerate(hrs):
        for k, hr_1 in enumerate(hr):
            if "+" not in hr_1:
                continue
            
            replace_str = "-00:00"
            if re.search(time_range_re, hr_1):
                replace_str = ""

            hours[i][j][k] = hours[i][j][k].replace("+", replace_str)

In [59]:
def get_all_days(days):
    short_days = ["Mon", "Tue", "Wed", "Th", "Fri", "Sa", "Su"]
    long_days = ["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday"]


    if "-" not in days:
        return [long_days[short_days.index(days)]]
    
    output = []

    day_range = days.split("-")
    i = short_days.index(day_range[0])
    output += [long_days[i]]
    while short_days[i] != day_range[1]:
        i = (i + 1) % 7
        output += [long_days[i]]
    return output
        
    
# Remove days off from opening hours
def remove_days_off(day_str, opening_hours):
    off_days = re.search(day_re, day_str)
    if not off_days:
        return
    
    off_days = off_days.group()
    days = get_all_days(off_days)
    
    for day in days:
        print(day)
        for i, hours in enumerate(reversed(opening_hours)):
            if hours['day'] == day:
                print(opening_hours[-i-1])




In [61]:
opening_hours = [{'day': 'monday', 'opens': '12:00', 'closes': '12:00'},{'day': 'monday', 'opens': '12:00', 'closes': '12:011'}, {'day': 'tuesday', 'opens': '12:00', 'closes': '12:00'}, {'day': 'wednesday', 'opens': '12:00', 'closes': '12:00'}, {'day': 'thursday', 'opens': '12:00', 'closes': '12:00'}, {'day': 'friday', 'opens': '12:00', 'closes': '12:00'}, {'day': 'saturday', 'opens': '11:00', 'closes': '12:00'}, {'day': 'sunday', 'opens': '11:00', 'closes': '12:00'}]
remove_days_off("Mon", opening_hours)

monday
{'day': 'monday', 'opens': '12:00', 'closes': '12:011'}
{'day': 'monday', 'opens': '12:00', 'closes': '12:00'}


In [16]:
for i, hrs in enumerate(hours):
    opening_hours = []
    for hr in hrs:
        for hr_1 in hr:
            if len(hr_1)>=3 and hr_1.endswith("off"):
                print(hr_1)
            if len(hr_1)>=6 and hr_1.endswith("closed"):
                print(hr_1)

Su off
Mon closed
Mon off
Tue off
Mon off
Tue off
Wed off
Su off
Su off
Tue off
Wed off
Tue off
Su off
Su off
Su off
closed
Jan 01 off
Nov Th[4] off
Dec 25 off
Su closed
Mon off
Su off
Mon off
Su off
Mon off
Tue off
Mon off
Mon off
Tue off
Su off
Su off
Mon off
Su closed
Su off
Su off
Mon-Tue off
Su closed
Su off
Sa off
Wed off
Tue off
Jan 01 off
Nov Th[4] off
Dec 25 off
Jan 1 off
Dec 25 off
Jan 01 off
Nov Th[4] off
Dec 25 off
Nov Th[4] off
Dec 25 off
Jan 01 off
Nov Th[4] off
Dec 25 off
Jan 01 off
Nov Th[4] off
Dec 25 off
Mon-Tue off
Mon off
Tue off
Mon closed
Su off
Mon off
Su off
Su closed
Nov Th[4] off
Su closed
Sa off
Mon-Wed off
Su off
Jan 1 off
Jul 4 off
Dec 25 off
Mon off
Su off
Jul 4 off
Dec 25 off
Nov-May off
Tue off
Wed off
Mon-Wed off
Wed off
Su off
Tue off
Su off
Tue off
Tue off
Mon off
Su off
Fri off
Su off
Mon-Tue off
Mon off
Tue off
Mon off
Mon off
Sa-Su off
Mon off
Su off
Oct-Mar off
Mon off
Mon off
Su off
Wed off
Mon off
Mon off
Tue off
Su off
Mon off
Tue off
Mon off
S

In [14]:
day_time_re = r"(Mon|Tue|Wed|Th|Fri|Sa|Su)?( *- *(Mon|Tue|Wed|Th|Fri|Sa|Su))? *([01]?\d|2[0-3]):([0-5]\d)( *- *([01]?\d|2[0-3]):([0-5]\d))"

no_re_count = 0 # No regex match count
exception_count = 0 # Exception count

for i, hrs in enumerate(hours):
    opening_hours = []
    for hr in hrs:
        for hr_1 in hr:
            if len(hr_1)>=3 and hr_1.endswith("off"):
                continue
            if len(hr_1)>=6 and hr_1.endswith("closed"):
                continue
            
            parse_string = re.search(day_time_re, hr_1)
            if not parse_string:
                print(hr_1, "no date")
                no_re_count += 1
                continue
            try:
                opening_hours += OpeningHours.parse(parse_string.group(), assume_type="24H").json()
            except:
                exception_count += 1

    print(original_hours[i])
    print(opening_hours)

Mo-Su 11:00-22:00
[{'day': 'monday', 'opens': '11:00', 'closes': '22:00'}, {'day': 'tuesday', 'opens': '11:00', 'closes': '22:00'}, {'day': 'wednesday', 'opens': '11:00', 'closes': '22:00'}, {'day': 'thursday', 'opens': '11:00', 'closes': '22:00'}, {'day': 'friday', 'opens': '11:00', 'closes': '22:00'}, {'day': 'saturday', 'opens': '11:00', 'closes': '22:00'}, {'day': 'sunday', 'opens': '11:00', 'closes': '22:00'}]
nan no date
nan
[]
Mo-Th 12:00-22:00; Fr-Sa 12:00-23:00; Su 12:00-22:00
[{'day': 'monday', 'opens': '12:00', 'closes': '22:00'}, {'day': 'tuesday', 'opens': '12:00', 'closes': '22:00'}, {'day': 'wednesday', 'opens': '12:00', 'closes': '22:00'}, {'day': 'thursday', 'opens': '12:00', 'closes': '22:00'}, {'day': 'friday', 'opens': '12:00', 'closes': '23:00'}, {'day': 'saturday', 'opens': '12:00', 'closes': '23:00'}, {'day': 'sunday', 'opens': '12:00', 'closes': '22:00'}]
Mo-Fr 09:00-13:00,15:00-18:00
[{'day': 'monday', 'opens': '9:00', 'closes': '13:00'}, {'day': 'tuesday', 'op

In [15]:
for hrs in hours:
    for hr in hrs:
        if re.search("PH", hr):
            print(hrs)

TypeError: expected string or bytes-like object, got 'list'

In [ ]:
exception_count

1

In [ ]:
no_re_count

22